# <span style="color:blue">FLASK</span>

Flask is a lightweight Web Server Gateway Interface application framework

https://flask.palletsprojects.com/en/2.0.x/

- Flask can offer resources via HTTP urls (routes).

Install flask library in a virtual environment:
```bash
(myvenv)% python -m pip install flask
```

## First example

Previous py file saved at path:

In [6]:
%%writefile cell1.py

port = 9999
from flask import Flask, escape, request

app = Flask(__name__)

@app.route('/')                           # wrappers - a function englobe another function: def route(function)
def hello():
    name = request.args.get("name", "World")
    return "Hello %s!"%name

if __name__ == '__main__':                # double_underscore, curso avanzado
    app.run(port=port, debug=True)

Overwriting cell1.py


In [7]:
%%system
pwd

['/home/andreuet/arquitectures/restful']

You can run in the terminal, the previous script (in that path):
```bash
 
```

Open a browser to get the data from the URL or use curl o wget

In [8]:
%%bash
curl http://localhost:9999/

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   232  100   232    0     0   226k      0 --:--:-- --:--:-- --:--:--  226k


**Note**: you can free the port killing the process
```bash
sudo kill -9 $(sudo lsof -t -i:9999)
```

## With Restful library we can expose services
https://dosideas.com/noticias/java/314-introduccion-a-los-servicios-web-restful

**Note** we need to install another library from flask

```bash
(myenv)$ pip install flask-restful
```

```python
#Basic skeleton of a restful api

from flask import Flask, escape, request
from flask_restful import Resource, Api

app = Flask(__name__)
api = Api(app)

# Resources
# ...
# Routes
# ...

if __name__ == '__main__':
    app.run(port='9999', debug=True)
```

The **resources** are clases with get and post functions 

```python
class Book(Resource):  # Class definition, extends Resource.class
  def get(self):       # self is the global variable of the class
    return {"book1":"HolaMundo"}
```

The **routes** are the path to the resource access 

```python
api.add_resource(Book, '/books')
```

In [ ]:
%%writefile cell2.py
from flask import Flask, escape, request
from flask_restful import Resource, Api

app = Flask(__name__)
api = Api(app)

class Book(Resource):    
  def get(self):         
    return {"book1":"HolaMundo"}  

api.add_resource(Book, '/books') 

if __name__ == '__main__':
    app.run(port='9999', debug=True)

Run cell2.py

In [ ]:
%%bash
curl http://localhost:9999/

In [ ]:
%%bash
curl http://localhost:9999/books

## <span style="color:red">A1. What TYPE of variable is it returning? </span> 

## <span style="color:blue">A1.Returns a string dictionary </span>

In [4]:
%%writefile cell3.py
from flask import Flask, escape, request
from flask_restful import Resource, Api
from flask_jsonpify import jsonify #install it

import random 

app = Flask(__name__)
api = Api(app)

class Book(Resource): 
  def get(self):       
    return jsonify(ids=[random.randint(0,100) for i in range(100)])

api.add_resource(Book, '/books') 

if __name__ == '__main__':
    app.run(port='9999', debug=True)

Overwriting cell3.py


## <span style="color:red">A2. What does the jsonify library? </span> 

## <span style="color:blue">A2. jsonify creates an object from Response's Class with the JSON representation of the given arguments. So, cell3.py return a JSON representation </span>

We can use the url path to interact with the rest api
```python
api.add_resource(GetBook, '/book/<book_id>')
```


We can use fake data to popoulate our book collection
```python
from lorem_text import lorem
import random
book_collection = [lorem.words(random.randint(4,10)) for i in range(100)]
```

In [1]:
from lorem_text import lorem #install it
import random
book_collection = [lorem.words(random.randint(4,10)) for i in range(100)]
print(book_collection)

ModuleNotFoundError: No module named 'lorem_text'

In [2]:
%%writefile cell4.py

from flask import Flask, escape, request
from flask_restful import Resource, Api
from flask_jsonpify import jsonify
import random 
from lorem_text import lorem

app = Flask(__name__)
api = Api(app)

book_collection = [lorem.words(random.randint(4,10)) for i in range(100)]

class Book(Resource):    #return a list of book-ids
  def get(self):       
    return jsonify(ids=list(range(len(book_collection))))

class GetBook(Resource): #return the title of a book
  def get(self,book_id):       
    return jsonify(id=book_id,title=book_collection[int(book_id)])

api.add_resource(Book, '/books') 
api.add_resource(GetBook, '/book/<book_id>') 

if __name__ == '__main__':
    app.run(port='9999', debug=True)

Writing cell4.py


In [ ]:
%%bash
curl http://localhost:9999/books

In [ ]:
%%bash
curl http://localhost:9999/book/7

## Second tests
Put methods

In [3]:
%%writefile cell5.py

from flask import Flask, escape, request
from flask_restful import Resource, Api
from flask_jsonpify import jsonify

import random 
from lorem_text import lorem

app = Flask(__name__)
api = Api(app)

book_collection = [lorem.words(random.randint(4,10)) for i in range(100)]

class Books(Resource): 
  def get(self):       
    return jsonify(ids=list(range(len(book_collection))))

class Book(Resource): 
  def get(self,book_id):       
    return jsonify(book_collection[int(book_id)])

  def post(self): #POST Method
    json_data = request.get_json(force=True) #obtain the data forcing JSON-type
    booktitle = json_data["title"]           #get the title key
    book_collection.append(booktitle)        #add the title in the collection
    new_id = len(book_collection)            #generate a new ID
    return jsonify(id=new_id)                #return the ID

api.add_resource(Books, '/books') 

api.add_resource(Book, '/book/<book_id>','/addbook/') #BOTH METHODS.

if __name__ == '__main__':
    app.run(port='9999', debug=True)

Writing cell5.py


To **generate a POST request** we have two basic options: REST debug extensions for web browers ie. YARC. or use CURL command:
```bash
curl -v -X POST http://localhost:9999/addbook/ -d '{"title":"Hello world book"}'
```

## <span style="color:red">A3. Activity </span> 
Design an API how manages a list of TODO tasks. The requirements are:


- Manage multiples lists. Lists by category. I.e. Shopping, Work, House,... 
- Get a list of all TODO items by default from all categories or specifing one category
- Add new item in a specific list
- Remove one item from a specific list


## <span style="color:red">A4. Activity </span> 
What are the differences between an RPC and a Restful service?

## All deliverables in the same GitHub project. Answers in one readme file of the main project. The main project is the Activity A3.